In [ ]:
!wget http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Cell_Phones_and_Accessories_5.json.gz

--2022-09-20 07:43:40--  http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Cell_Phones_and_Accessories_5.json.gz
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80
Connecting to snap.stanford.edu (snap.stanford.edu)|171.64.75.80|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 45409631 (43M) [application/x-gzip]
Saving to: ‘reviews_Cell_Phones_and_Accessories_5.json.gz.2’

reviews_Cell_Phones 100%[===================>]  43.31M  29.2MB/s    in 1.5s    

2022-09-20 07:43:42 (29.2 MB/s) - ‘reviews_Cell_Phones_and_Accessories_5.json.gz.2’ saved [45409631/45409631]



In [ ]:
import pandas as pd
import gzip

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df = getDF('reviews_Cell_Phones_and_Accessories_5.json.gz')

In [ ]:
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A30TL5EWN6DFXT,120401325X,christina,"[0, 0]",They look good and stick good! I just don't li...,4.0,Looks Good,1400630400,"05 21, 2014"
1,ASY55RVNIL0UD,120401325X,emily l.,"[0, 0]",These stickers work like the review says they ...,5.0,Really great product.,1389657600,"01 14, 2014"
2,A2TMXE2AFO7ONB,120401325X,Erica,"[0, 0]",These are awesome and make my phone look so st...,5.0,LOVE LOVE LOVE,1403740800,"06 26, 2014"
3,AWJ0WZQYMYFQ4,120401325X,JM,"[4, 4]",Item arrived in great time and was in perfect ...,4.0,Cute!,1382313600,"10 21, 2013"
4,ATX7CZYFXI1KW,120401325X,patrice m rogoza,"[2, 3]","awesome! stays on, and looks great. can be use...",5.0,leopard home button sticker for iphone 4s,1359849600,"02 3, 2013"


In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize, sent_tokenize
nltk.download('stopwords')
from nltk.corpus import stopwords 
from nltk.stem import PorterStemmer

stemmer = PorterStemmer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
df['reviewText'] = df['reviewText'].str.lower()

In [ ]:
def identify_tokens(row):
    review = row['reviewText']
    tokens = word_tokenize(review)
    # taken only words (not punctuation)
    token_words = [w for w in tokens if w.isalpha()]
    return token_words

df['words'] = df.apply(identify_tokens, axis=1)

In [ ]:
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,words
0,A30TL5EWN6DFXT,120401325X,christina,"[0, 0]",they look good and stick good! i just don't li...,4.0,Looks Good,1400630400,"05 21, 2014","[they, look, good, and, stick, good, i, just, ..."
1,ASY55RVNIL0UD,120401325X,emily l.,"[0, 0]",these stickers work like the review says they ...,5.0,Really great product.,1389657600,"01 14, 2014","[these, stickers, work, like, the, review, say..."
2,A2TMXE2AFO7ONB,120401325X,Erica,"[0, 0]",these are awesome and make my phone look so st...,5.0,LOVE LOVE LOVE,1403740800,"06 26, 2014","[these, are, awesome, and, make, my, phone, lo..."
3,AWJ0WZQYMYFQ4,120401325X,JM,"[4, 4]",item arrived in great time and was in perfect ...,4.0,Cute!,1382313600,"10 21, 2013","[item, arrived, in, great, time, and, was, in,..."
4,ATX7CZYFXI1KW,120401325X,patrice m rogoza,"[2, 3]","awesome! stays on, and looks great. can be use...",5.0,leopard home button sticker for iphone 4s,1359849600,"02 3, 2013","[awesome, stays, on, and, looks, great, can, b..."


In [ ]:
def stem_list(row):
    my_list = row['words']
    stemmed_list = [stemmer.stem(word) for word in my_list]
    return (stemmed_list)

df['stemmed_words'] = df.apply(stem_list, axis=1)

In [ ]:
stops = set(stopwords.words("english"))                  

def remove_stops(row):
    my_list = row['stemmed_words']
    meaningful_words = [w for w in my_list if not w in stops]
    return (meaningful_words)

df['stem_meaningful'] = df.apply(remove_stops, axis=1)

In [ ]:
!pip install gensim

In [ ]:
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,words,stemmed_words,stem_meaningful
0,A30TL5EWN6DFXT,120401325X,christina,"[0, 0]",they look good and stick good! i just don't li...,4.0,Looks Good,1400630400,"05 21, 2014","[they, look, good, and, stick, good, i, just, ...","[they, look, good, and, stick, good, i, just, ...","[look, good, stick, good, like, round, shape, ..."
1,ASY55RVNIL0UD,120401325X,emily l.,"[0, 0]",these stickers work like the review says they ...,5.0,Really great product.,1389657600,"01 14, 2014","[these, stickers, work, like, the, review, say...","[these, sticker, work, like, the, review, say,...","[sticker, work, like, review, say, stick, grea..."
2,A2TMXE2AFO7ONB,120401325X,Erica,"[0, 0]",these are awesome and make my phone look so st...,5.0,LOVE LOVE LOVE,1403740800,"06 26, 2014","[these, are, awesome, and, make, my, phone, lo...","[these, are, awesom, and, make, my, phone, loo...","[awesom, make, phone, look, stylish, onli, use..."
3,AWJ0WZQYMYFQ4,120401325X,JM,"[4, 4]",item arrived in great time and was in perfect ...,4.0,Cute!,1382313600,"10 21, 2013","[item, arrived, in, great, time, and, was, in,...","[item, arriv, in, great, time, and, wa, in, pe...","[item, arriv, great, time, wa, perfect, condit..."
4,ATX7CZYFXI1KW,120401325X,patrice m rogoza,"[2, 3]","awesome! stays on, and looks great. can be use...",5.0,leopard home button sticker for iphone 4s,1359849600,"02 3, 2013","[awesome, stays, on, and, looks, great, can, b...","[awesom, stay, on, and, look, great, can, be, ...","[awesom, stay, look, great, use, multipl, appl..."


In [ ]:
def rejoin_words(row):
    my_list = row['stem_meaningful']
    joined_words = ( " ".join(my_list))
    return joined_words

df['processed'] = df.apply(rejoin_words, axis=1)
review_list = df['processed'].loc[0:14999].to_numpy()
review_labels = df['overall'].loc[0:14999].to_numpy()

In [ ]:
xx = review_list.tolist()

In [ ]:
 import logging
 import os
 import sys
 import gensim
 from gensim.models import Word2Vec
 from gensim.models import word2vec
 from gensim.models.word2vec import LineSentence

 
 # load data
 sentences = xx
 # train model
#  word2vec.Word2Vec([s.encode('utf-8').split() for s in sentences], size=100, window=5, min_count=5, workers=4)
 model = gensim.models.Word2Vec([s.split() for s in sentences], size=200, min_count=1)
#  model = gensim.models.Word2Vec([sentences],min_count=1,size=32)
 # save model
 model.save('word2vec.model')
 # load model
 model = gensim.models.Word2Vec.load('word2vec.model')
 # look up top 6 words similar to 'good'
y1 = model.most_similar("stick", topn=6)
print(u"stick\n")
for item in y1:
    print(item[0], item[1])  

stick

unclip 0.9044105410575867
wrap 0.8917160034179688
arm 0.8844923377037048
bend 0.8813488483428955
sticki 0.8808974027633667
stuck 0.8803854584693909


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).


In [ ]:
!pip install googletrans==4.0.0rc1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from googletrans import Translator
translator = Translator()
list_news = ['आखिर वॉशरूम में कैसे बना MMS? सामने आईं ये बड़ी खामियां', "नौकरीपेशा लोगों को वर्ल्ड बैंक की रिपोर्ट क्यों परेशान कर रही, 5 पॉइंट में समझें"]
sentences = []
for i in list_news:
  result = translator.translate(i)
  sentences.append(result.text)

In [ ]:
sentences

['After all, how did MMS make in washroom?These big flaws surfaced',
 'Why are the reports of world bank bothering employed people, understand in 5 points']

In [ ]:
def replace_oovs(vocab, data, unk="<unk>"):
    """
    vocab: set of tokens
    data: list of lists, i.e. list of sentences, which are lists of tokens
    unk: token to replace tokens which are not in the vocabulary
    
    This function replaces all tokens not in the vocabulary with the unknown token
    """
    data_oovs_replaced = list(data)
    for i, sentence in enumerate(data_oovs_replaced):
        for j, token in enumerate(sentence):
            if token not in vocab:
                data_oovs_replaced[i][j] = unk

    return data_oovs_replaced

In [ ]:
from collections import Counter
import itertools
unigram_counts = Counter(itertools.chain.from_iterable(sentences)).most_common()
vocab = set(word for word, freq in unigram_counts[:1000]) | {"</s>", "<unk>"}
sen = replace_oovs(vocab, sentences)

In [ ]:
model = gensim.models.Word2Vec([s.split() for s in sen], size=200, min_count=1)

model.save('word2vec_news.model')
model = gensim.models.Word2Vec.load('word2vec_news.model')
y1 = model.most_similar("make", topn=6)
print(u"make\n")
for item in y1:
    print(item[0], item[1])  

make

did 0.11921481788158417
reports 0.11648842692375183
big 0.07427027821540833
all, 0.06034557521343231
how 0.05450060963630676
flaws 0.036223746836185455


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """


In [ ]:
len(df["asin"].unique())

10429

Recommendation System

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split

In [ ]:
def split_vals(a,n): 
  return a[:n].copy(), a[n:].copy()

n_valid = 5000  # same as Kaggle's test set size
n_trn = 15000 - n_valid
# raw_train, raw_valid = split_vals(df_raw, n_trn)
train_data, test_data = split_vals(review_list, n_trn)
train_labels, test_labels = split_vals(review_labels, n_trn)

train_data.shape, train_labels.shape, test_data.shape

((10000,), (10000,), (5000,))

In [ ]:
counter = CountVectorizer()
counter.fit(train_data)
len(counter.vocabulary_)

12327

In [ ]:
training_counts = counter.transform(train_data)
test_counts = counter.transform(test_data)
training_counts.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
classifier = MultinomialNB()
classifier.fit(training_counts, train_labels)
print(classifier.predict(test_counts))
print(classifier.predict_proba(test_counts))

[5. 5. 5. ... 5. 4. 5.]
[[9.14713015e-16 9.79757955e-22 7.24747880e-16 3.32469456e-02
  9.66753054e-01]
 [7.36017285e-05 1.64802081e-06 3.42094234e-03 1.76929090e-01
  8.19574718e-01]
 [5.00312170e-02 1.19548212e-01 1.26605758e-01 3.03755192e-01
  4.00059622e-01]
 ...
 [1.18089072e-03 3.11187026e-04 4.44430590e-02 2.48390868e-01
  7.05673995e-01]
 [1.11175090e-02 2.79236010e-02 1.60084768e-01 4.52473508e-01
  3.48400613e-01]
 [3.42249846e-01 6.19964343e-04 7.93860596e-02 1.26489526e-02
  5.65095178e-01]]


In [ ]:
print(classifier.score(test_counts, test_labels))

0.5538
